In [4]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver

import numpy as np
import string
import datetime
import time
import csv
import ast

In [5]:
# global variables
%store -r playersToIds
%store -r idsToPlayers
%store -r datesDict
%store -r statsNBAMap

In [6]:
def readData():
    with open('allplayers.csv', mode='r') as infile:
        reader = csv.reader(infile)
        with open('allreader.csv', mode='w') as outfile:
            writer = csv.writer(outfile)
            mydict = {rows[0]:rows[1] for rows in reader}

    for name in mydict:
        val = mydict[name]
        mydict[name] = eval(val)

    return mydict

def setupHeadlessChrome():
    options = webdriver.ChromeOptions() # don't open URL window
    options.add_argument('--headless')
    prefs = {"profile.managed_default_content_settings.images": 2, 'disk-cache-size': 4096 }
    options.add_experimental_option("prefs", prefs)
    return options

In [7]:
allPlayers = readData()

In [8]:
allPlayers['Andrew Wiggins']

{'InjResp': (1.5800000000000005,
  {'2018-19': [[-7.9,
     [['Los Angeles Lakers', '.375'],
      ['Milwaukee Bucks', '1.000'],
      ['Toronto Raptors', '.875'],
      ['Utah Jazz', '.571']],
     datetime.datetime(2018, 10, 24, 0, 0),
     datetime.datetime(2018, 10, 31, 0, 0),
     4],
    [8.9,
     [['Houston Rockets', '.569'], ['LA Clippers', '.542']],
     datetime.datetime(2019, 2, 11, 0, 0),
     datetime.datetime(2019, 2, 21, 0, 0),
     2]],
   '2019-20': [[7.4,
     [['Brooklyn Nets', '.471'],
      ['Cleveland Cavaliers', '.278'],
      ['Golden State Warriors', '.243'],
      ['Milwaukee Bucks', '.865']],
     datetime.datetime(2019, 12, 30, 0, 0),
     datetime.datetime(2020, 1, 5, 0, 0),
     4]]}),
 'ActResp': (-3.038211382113821,
  {'2015-16': [[2.3,
     [['Atlanta Hawks', '.778'],
      ['Charlotte Hornets', '.429'],
      ['Chicago Bulls', '.625'],
      ['Denver Nuggets', '.429'],
      ['Los Angeles Lakers', '.143'],
      ['Miami Heat', '.625'],
      ['Portlan

In [9]:
url = "https://www.basketball-reference.com/leagues/NBA_2019.html"
options = setupHeadlessChrome()
    
driver = webdriver.Chrome(options=options) # used to view generated JavaScript

driver.get(url) 
html = driver.page_source

In [10]:
teamTable = driver.find_elements_by_id("team-stats-per_game")
teamRows = teamTable[0].find_elements_by_tag_name('tbody')[0].find_elements_by_tag_name('a')

for i in range(0, len(teamRows)):
    teamRow = teamRows[i]
    link = teamRow.get_attribute('href')
    print(link)

https://www.basketball-reference.com/teams/MIL/2019.html
https://www.basketball-reference.com/teams/GSW/2019.html
https://www.basketball-reference.com/teams/NOP/2019.html
https://www.basketball-reference.com/teams/PHI/2019.html
https://www.basketball-reference.com/teams/LAC/2019.html
https://www.basketball-reference.com/teams/POR/2019.html
https://www.basketball-reference.com/teams/OKC/2019.html
https://www.basketball-reference.com/teams/TOR/2019.html
https://www.basketball-reference.com/teams/SAC/2019.html
https://www.basketball-reference.com/teams/WAS/2019.html
https://www.basketball-reference.com/teams/HOU/2019.html
https://www.basketball-reference.com/teams/ATL/2019.html
https://www.basketball-reference.com/teams/MIN/2019.html
https://www.basketball-reference.com/teams/BOS/2019.html
https://www.basketball-reference.com/teams/BRK/2019.html
https://www.basketball-reference.com/teams/LAL/2019.html
https://www.basketball-reference.com/teams/UTA/2019.html
https://www.basketball-referenc

In [75]:
#['InjResp', 'ActResp', 'NetRatings', 'TeammateTimes', 'Teams', 
#'PositionRatios', 'Trades', 'MidseasonTradeRatings', 'PlusMinus', 'PointStreaks', 'AssistStreaks']
def createPlayerRow(teamUrl):
    html = urlopen(teamUrl)
    soup = BeautifulSoup(html, features="lxml")
    year = soup.find(id='meta').find('span').text
    players = soup.findAll('tr')
    teammates = {}
    for x in range(1, len(players)):
        playerRow = players[x]
        playerName = playerRow.find('a').text
        teammates[playerName] = None
        
    for i in range(1, len(players)):
        playerRow = players[i]
        playerName = playerRow.find('a').text
        if playerName not in allPlayers:
            playerName += " Jr."
        playerData = allPlayers[playerName]
        print(playerName)

        indivRecord = getIndivRecord(playerData, year, playerName, teammates)
        print(indivRecord)
        teammates[playerName] = indivRecord
    
# def getCombinedRecord(teammates):
#     combinedRecord = []
#     for teammate, entries in teammates.items():
        
        
def getIndivRecord(playerData, year, playerName, teammates):
    injResp = getInjRespFeature(playerData, year, True) # [weightedRating, totalGames]
    actResp = getInjRespFeature(playerData, year, False) # [weightedRating, totalGames]
    netRatingsList = getNetRatingsFeature(playerData, year) # ['Overall', '0 Days Rest',...]
    teammateTimes = getCurrentTeammateTimes(playerData, playerName, year, teammates) # Total time
    posRatios = getPositionRataioFeature(playerData, year) # [PG, SG, SF, PF, C]
    numTrades = getNumTrades(playerData, year) # number
    numMidTrades = getNumMidseasonTrades(playerData, year) # number
    midTradeResp = getMidseasonTradeFeature(playerData, year) # [rating before, rating after]
    plusMinus = getPlusMinusFeature(playerData, year) # [on court, on/off]
    pointStreaks = getPointStreaks(playerData, year, True) # [avgStreakLength, totalNumStreaks, avgStreakLengthPerYear, avgStreakCountPerYear]
    assistStreaks = getPointStreaks(playerData, year, False) # [avgStreakLength, totalNumStreaks, avgStreakLengthPerYear, avgStreakCountPerYear]

    indivRecord = []
    indivRecord.extend(injResp)
    indivRecord.extend(actResp)
    indivRecord.extend(netRatingsList)
    indivRecord.append(teammateTimes)
    indivRecord.extend(posRatios)
    indivRecord.append(numTrades)
    indivRecord.append(numMidTrades)
    indivRecord.extend(plusMinus)
    indivRecord.extend(pointStreaks)
    indivRecord.extend(assistStreaks)
    
    return indivRecord
        
def getInjRespFeature(playerData, year, isInjury):
    injResp = playerData['InjResp'] if isInjury else playerData['ActResp']
    return injRespFeatureHelper(injResp, year)

def injRespFeatureHelper(injResp, year):
    injuryRatings = []
    totalGames = 0
    weightedRating = 0
    
    if injResp == 0: # no injuries
        return [0, 0]
    for season, entries in injResp[1].items():
        if checkSeason(year, season):
            for injury in entries:
                rating = injury[0]
                numGames = len(injury[1])
                totalGames += numGames
                injuryRatings.append((rating, numGames))
    for inj in injuryRatings:
        weightedRating += inj[0]*(inj[1]/totalGames)
    return [weightedRating, totalGames]

def getNetRatingsFeature(playerData, year):
    netRatings = playerData["NetRatings"]
    count = 0
    
    restMap = {'Overall': 0, '0 Days Rest': 0, '1 Days Rest': 0, '2 Days Rest': 0,\
               '3 Days Rest': 0, '4 Days Rest': 0, '5 Days Rest': 0, '6+ Days Rest': 0}
    for season, entries in netRatings.items():
        perYear = {'Overall': 0, '0 Days Rest': 0, '1 Days Rest': 0, '2 Days Rest': 0,\
                   '3 Days Rest': 0, '4 Days Rest': 0, '5 Days Rest': 0, '6+ Days Rest': 0}
        if checkSeason(year, season):
            count += 1
            for team, ratings in entries.items():
                for restType, value in ratings.items():
                    perYear[restType] += float(value) * 1/(len(entries))
        for entry, val in perYear.items():
            restMap[entry] += val
    
    result = [restMap['Overall']/count, restMap['0 Days Rest']/count,\
              restMap['1 Days Rest']/count, restMap['2 Days Rest']/count,\
              restMap['3 Days Rest']/count, restMap['4 Days Rest']/count,\
              restMap['5 Days Rest']/count, restMap['6+ Days Rest']/count]
    return result # size-8 list

def getCurrentTeammateTimes(playerData, playerName, year, teammates):
    teammateTimes = playerData['TeammateTimes']
    totalTime = 0
    for season, entries in teammateTimes[1].items():
        if checkSeason(year, season):
            for oldTeammate in entries:
                if oldTeammate in teammates or juniorCheck(oldTeammate, teammates):
                    totalTime += entries[oldTeammate].total_seconds()
    return totalTime

def getPositionRataioFeature(playerData, year):
    posRatios = playerData['PositionRatios']
    res = [0, 0, 0, 0, 0]
    count = 0
    for season, entries in posRatios.items():
        if season != "Career" and checkSeason(year, season):
            count += 1
            for position, ratio in entries.items():
                res[int(position)-1] += ratio
    if count == 0:
        return res
    for i in range(0, len(res)):
        res[i] = res[i]/count
    return res

def getNumTrades(playerData, year):
    trades = playerData['Trades']
    return len(trades[0])

def getNumMidseasonTrades(playerData, year):
    trades = playerData['Trades']
    return len(trades[1])

def getMidseasonTradeFeature(playerData, year):
    weekBefore = 0
    weekAfter = 0
    count = 0
    for tradeDate, ratings in playerData['MidseasonTradeRatings'].items():
        season = getYear(tradeDate)
        if checkSeason(year, season):
            weekBefore += ratings[0]
            weekAfter += ratings[1]
            count += 1
    if count == 0:
        return [0, 0]
    return [weekBefore/count, weekAfter/count]

def getPlusMinusFeature(playerData, year):
    count = 0
    onCourt = 0
    onOff = 0
    plusMinusRatings = playerData['PlusMinus']
    for season, entries in plusMinusRatings.items():
        if season != "Career" and checkSeason(year, season):
            count += 1
            onCourt += float(entries[0])
            onOff += float(entries[1])
    if count == 0:
        return [0, 0]
    return [onCourt/count, onOff/count]

def getPointStreaks(playerData, year, isPoints):
    streakCount = 0
    totalStreak = 0.0
    
    yearlyAverages = []
    yearlyCounts = []
    
    pointStreaks = playerData['PointStreaks'] if isPoints else playerData['AssistStreaks']
    for season, entries in pointStreaks.items():
        if checkSeason(year, season):
            yearCount = 0
            yearStreak = 0
            for streak in entries:
                yearCount += 1
                yearStreak += float(streak[2])
            
            yearlyAverages.append(yearStreak/yearCount if yearCount != 0 else 0)
            yearlyCounts.append(yearCount)
            
            streakCount += yearCount
            totalStreak += yearStreak
    
    avgStreakLength = totalStreak/streakCount if streakCount != 0 else 0
    totalNumStreaks = sum(yearlyCounts)
    
    avgStreakLengthPerYear = sum(yearlyAverages)/len(yearlyAverages) if len(yearlyAverages) != 0 else 0
    avgStreakCountPerYear = sum(yearlyCounts)/len(yearlyCounts) if len(yearlyCounts) != 0 else 0
    
    return [avgStreakLength, totalNumStreaks, avgStreakLengthPerYear, avgStreakCountPerYear]
                

def juniorCheck(playerName, teammates): #Tim Hardaway Jr. listed as Tim Hardaway on Basketball Reference
    return playerName + " Jr." in teammates
    
def checkSeason(year, season):
    yIndex = year.index('-')
    sIndex = season.index('-')
    return int(season[:sIndex]) <= int(year[:yIndex])

def getYear(date):
    potentialYears = getPotentialTradeYears(date.year)
    firstStart = datetime.datetime.strptime(datesDict[potentialYears[0]][0], '%a, %b %d, %Y')
    firstEnd = datetime.datetime.strptime(datesDict[potentialYears[0]][1], '%a, %b %d, %Y')
    secondStart = datetime.datetime.strptime(datesDict[potentialYears[1]][0], '%a, %b %d, %Y')
    secondEnd = datetime.datetime.strptime(datesDict[potentialYears[1]][1], '%a, %b %d, %Y')

    if date >= firstStart and date <= firstEnd:
        year = potentialYears[0]
    elif date >= secondStart and date <= secondEnd:
        year = potentialYears[1] 
    return year

def getPotentialTradeYears(year):
    options = []
    options.append(str(year-1) + '-' + str(year)[2:])
    options.append(str(year) + '-' + str((year+1))[2:])
    if options[1] == "2020-21":
        options[1] = "2019-20"
    return options

In [76]:
createPlayerRow("https://www.basketball-reference.com/teams/DAL/2019.html")

Kostas Antetokounmpo
[3.251724137931034, 29, -3.8, 30, -1.5, -8.3, -0.5, 1.0, 14.6, -22.1, 0.0, -9.7, 2536.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1, 0, 18.2, 20.0, 0, 0, 0.0, 0.0, 0, 0, 0.0, 0.0]
J.J. Barea
[2.6804597701149424, 87, 0.7275193798449611, 516, 1.1692307692307697, -1.2384615384615385, 1.9692307692307693, 1.1, 3.96923076923077, 5.315384615384615, 1.4923076923076921, -2.5999999999999996, 1073649.0, 0.9553846153846154, 0.04538461538461539, 0.0, 0.0, 0.0, 0, 0, -0.11538461538461545, -1.676923076923076, 2.3333333333333335, 6, 0.6923076923076923, 0.46153846153846156, 3.287878787878788, 66, 3.049664224664225, 5.076923076923077]
Harrison Barnes
[4.333333333333333, 18, 0.8999999999999999, 69, 2.7214285714285715, 0.13571428571428587, 3.185714285714286, 3.1000000000000005, 7.935714285714285, -0.6500000000000001, 2.757142857142857, -7.864285714285714, 1132047.0, 0.0, 0.009999999999999998, 0.5385714285714286, 0.4428571428571429, 0.0071428571428571435, 1, 1, 2.0142857142857142, -2.5, 2.36, 25, 1.85